In [16]:
import torch
from safetensors import safe_open

In [21]:
file_path = '/mnt/LLM/hub/models--meta-llama--Meta-Llama-3-70B/blobs/fbc173eeb6cb16d1415f19e2f6590e1a1bfe55a483ae8c66e8e6ce95215f21b5'
with safe_open(file_path, framework="pt", device="cpu") as f:
    # Get tensor names in this file
    tensor_names = f.keys()
    # Load just one tensor
    single_tensor = f.get_tensor(list(tensor_names)[1])

In [25]:
single_tensor.abs().mean()

tensor(0.0098, dtype=torch.bfloat16)

In [26]:
(1 ** 2 / (100_000_000_000)) ** 0.5

3.162277660168379e-06

In [32]:
(torch.tensor(1e-6, dtype=torch.bfloat16) + 1e-8).float() - (torch.tensor(1e-6, dtype=torch.bfloat16)).float()

tensor(7.4506e-09)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import torch
device = "cuda:1"

In [6]:
from transformers import LlamaConfig, AutoModel, AutoModelForCausalLM
import json

# Method 1
with open('configs/llama_60m.json', 'r') as file:
    config_dict = json.load(file)
    

/home/zmushko-fa/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import inspect
args = inspect.signature(LlamaConfig.__init__).parameters

In [8]:
args_list = list(arg for arg in args)
for arg in args:
    if arg not in config_dict.keys():
        print(arg)

self
num_key_value_heads
max_position_embeddings
pretraining_tp
tie_word_embeddings
rope_theta
rope_scaling
attention_bias
attention_dropout
mlp_bias
head_dim
kwargs


In [9]:
args_list = list(arg for arg in args)
new_config_dict = {}
for k in config_dict:
    if k not in args_list:
        print(k, config_dict[k])
    else:
        new_config_dict[k] = config_dict[k]

architectures ['LLaMAForCausalLM']
max_sequence_length 1024
model_type llama
transformers_version 4.28.1


In [10]:
from utils.modeling_llama import LlamaForCausalLM as CustomLlama
from transformers import AutoConfig


config = LlamaConfig(**new_config_dict, num_key_value_heads=new_config_dict["num_attention_heads"])
torch.manual_seed(0)
model = AutoModelForCausalLM.from_config(config).to(device)

auto_config = AutoConfig.from_pretrained("configs/llama_60m.json")
torch.manual_seed(0)
model_auto_config = AutoModelForCausalLM.from_config(auto_config).to(device)

for arg in model.config:
    if getattr(model.config, arg) != getattr(model_auto_config.config, arg):
        print(arg, "model:", getattr(model.config, arg), "model auto:", getattr(model_auto_config.config, arg))

torch.manual_seed(0)
custom_model = CustomLlama(auto_config).to(device)

/home/zmushko-fa/miniconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(
LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class

architectures model: None model auto: ['LLaMAForCausalLM']
_name_or_path model:  model auto: configs/llama_60m.json
transformers_version model: None model auto: 4.28.1


In [12]:
model.config.hidden_size

512

In [13]:
model_auto_config.config._attn_implementation_internal

'sdpa'

In [55]:
for (name1, p1), (name2, p2), (name3, p3) in zip(model_auto_config.named_parameters(), model.named_parameters(), custom_model.named_parameters()):
    assert name1 == name2 == name3
    assert p1.data.eq(p2.data).all() and p1.data.eq(p3.data).all()

In [57]:
torch.manual_seed(0)
input_ids = torch.randint(0, 32000, (2, 10), dtype=torch.long, device=device)
logits = model(input_ids).logits
logits2 = model(input_ids).logits
logits_custom = custom_model(input_ids).logits
logits_auto_config = model_auto_config(input_ids).logits

logits.eq(logits_custom).all(), logits.eq(logits_auto_config).all(), logits_custom.eq(logits_auto_config).all()

(tensor(False, device='cuda:1'),
 tensor(True, device='cuda:1'),
 tensor(False, device='cuda:1'))

In [28]:
(logits - logits_c).abs().max()

tensor(8.9407e-07, device='cuda:1', grad_fn=<MaxBackward1>)

In [26]:
logits_c

tensor([[[-0.1785, -0.4982,  0.7107,  ...,  0.4550,  0.0583,  0.1518],
         [-0.1425, -0.4003,  0.9012,  ...,  0.1133,  0.2125,  0.1764],
         [-0.0111, -0.3142,  0.6533,  ...,  0.0941,  0.3157,  0.2288],
         ...,
         [-0.1489, -0.4574,  0.5064,  ..., -0.1208,  0.3462,  0.2093],
         [-0.1052, -0.7117,  0.4115,  ...,  0.0022,  0.4919,  0.3694],
         [ 0.0518, -0.8056,  0.4821,  ..., -0.1090,  0.5137,  0.3687]],

        [[ 0.3210,  0.2530,  0.5245,  ...,  0.2575,  0.0839,  0.2333],
         [ 0.4388,  0.3758,  0.5472,  ...,  0.0641, -0.3066,  0.0449],
         [ 0.1868,  0.1871,  0.4683,  ...,  0.1585, -0.7356,  0.1738],
         ...,
         [ 0.5923,  0.1250,  0.3390,  ...,  0.0136, -0.7442,  0.1801],
         [ 0.5866,  0.1197,  0.2293,  ...,  0.1272, -0.6348,  0.2422],
         [ 0.6541,  0.2405,  0.1744,  ...,  0.0930, -0.6245,  0.1853]]],
       device='cuda:1', grad_fn=<UnsafeViewBackward0>)

In [94]:
hs = model.model.layers[0].input_layernorm(hidden_states)
out = model.model.layers[0].self_attn(hidden_states=hs, attention_mask=attention_mask, position_ids=position_ids, output_attentions=output_attentions)
out_c = custom_model.model.layers[0].self_attn(hidden_states=hs, attention_mask=attention_mask, position_ids=position_ids, output_attentions=output_attentions)
# out = model.model.layers[0].mlp(hs)
# out_c = custom_model.model.layers[0].mlp(hs)

In [95]:
out[0].eq(out_c[0]), out[0].eq(out_c[0]).all(), out[0].allclose(out_c[0])

(tensor([[[ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True, False,  True,  ..., False, False, False],
          ...,
          [ True,  True,  True,  ..., False, False, False],
          [ True,  True, False,  ..., False,  True, False],
          [False,  True,  True,  ..., False, False, False]],
 
         [[ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ..., False,  True, False],
          [ True,  True, False,  ...,  True, False,  True],
          ...,
          [False, False,  True,  ..., False, False,  True],
          [ True, False,  True,  ...,  True, False, False],
          [ True, False, False,  ..., False, False, False]]], device='cuda:1'),
 tensor(False, device='cuda:1'),
 False)

In [99]:
out = model.model.layers[0].mlp(hs)
out_c = custom_model.model.layers[0].mlp(hs)

In [107]:
mlp, mlp_c = model.model.layers[0].mlp, custom_model.model.layers[0].mlp

In [ ]:
mlp.act_fn, 

SiLU()

In [113]:
mlp.gate_proj.weight.eq(mlp_c.gate_proj.weight).all(), mlp.up_proj.weight.eq(mlp_c.up_proj.weight).all(), mlp.down_proj.weight.eq(mlp_c.down_proj.weight).all()

(tensor(True, device='cuda:1'),
 tensor(False, device='cuda:1'),
 tensor(False, device='cuda:1'))

In [75]:
from copy import deepcopy

def foo(parent_name, parent_module, parent_name_c, parent_module_c, input):
    print(parent_name)
    assert parent_name == parent_name_c, (name, name_c)
    input = input.clone()
    input_c = input.clone()
    for (child_name, child_module), (child_name_c, child_module_c) in zip(parent_module.named_children(), parent_module_c.named_children()):
        print(child_name, child_name_c)
        if child_name != child_name_c:
            foo(child_name, child_module, child_name_c, child_module_c, input)
        
        print("isinstance(child_module, torch.nn.ModuleList)", isinstance(child_module, torch.nn.ModuleList))
        if not isinstance(child_module, torch.nn.ModuleList):
            x = child_module(input)
            x_c = child_module_c(input_c)
        else:
            for i, (layer, layer_c) in enumerate(zip(child_module, child_module_c)):
                print(i)
                x = layer(input)
                x_c = layer_c(input_c)
                check, check_c = x, x_c
                if not isinstance(check, torch.Tensor):
                    check = check[0].clone()
                while not isinstance(check_c, torch.Tensor):
                    check_c = check_c[0].clone()
                if not check.eq(check_c).all():
                    foo(f"layer_{i}", layer, f"layer_{i}", layer_c, input)
                    return
                input, input_c = x, x_c
        check, check_c = x, x_c
        if not isinstance(check, torch.Tensor):
            check = check[0].clone()
        while not isinstance(check_c, torch.Tensor):
            check_c = check_c[0].clone()
        if not check.eq(check_c).all():
            foo(child_name, child_module, child_name_c, child_module_c, input)
            return
        print(x, x_c)
        input, input_c = x, x_c

foo("model", model.to("cuda:1"), "model", custom_model.to("cuda:1"), input_ids.to("cuda:1"))

model
model model
isinstance(child_module, torch.nn.ModuleList) False
model
embed_tokens embed_tokens
isinstance(child_module, torch.nn.ModuleList) False
tensor([[[-2.1211e-02,  1.5187e-02,  7.8467e-03,  ...,  7.0287e-04,
          -4.3865e-02,  1.3202e-02],
         [-1.1476e-02,  3.0487e-02, -3.3294e-02,  ...,  6.2483e-03,
          -1.9375e-02, -5.4145e-03],
         [ 4.3534e-03, -6.7081e-03, -1.1448e-02,  ..., -5.8181e-03,
           2.4763e-02, -2.1937e-02],
         ...,
         [-5.0058e-04,  1.4783e-02, -5.9794e-03,  ...,  2.5936e-02,
          -9.3622e-03, -6.1800e-03],
         [ 9.7384e-04,  2.2270e-02, -1.5616e-02,  ..., -1.6225e-02,
           1.2520e-02, -8.5498e-03],
         [-1.1799e-02,  8.0785e-05,  5.6678e-03,  ...,  2.1458e-02,
           2.2168e-03,  1.9806e-02]],

        [[-2.9679e-02, -1.7686e-02,  8.6890e-04,  ..., -2.4632e-02,
           3.5928e-03, -1.0772e-02],
         [-1.3895e-02, -1.0822e-02,  2.5041e-02,  ...,  1.0523e-02,
          -2.1157e-02,  2.4

AttributeError: 'NoneType' object has no attribute 'shape'

In [79]:
from copy import deepcopy

def foo(parent_name, parent_module, parent_name_c, parent_module_c):
    assert parent_name == parent_name_c, (parent_name, parent_name_c)
    print(f'\nparent {parent_name}')
    for (child_name, child_module), (child_name_c, child_module_c) in zip(parent_module.named_children(), parent_module_c.named_children()):
        print(child_name, child_name_c)
    for (child_name, child_module), (child_name_c, child_module_c) in zip(parent_module.named_children(), parent_module_c.named_children()):
        foo(child_name, child_module, child_name_c, child_module_c)
        
foo("model", model.to("cuda:1"), "model", custom_model.to("cuda:1"))


parent model
model model
lm_head lm_head

parent model
embed_tokens embed_tokens
layers layers
norm norm

parent embed_tokens

parent layers
0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7

parent 0
self_attn self_attn
mlp mlp
input_layernorm input_layernorm
post_attention_layernorm post_attention_layernorm

parent self_attn
q_proj q_proj
k_proj k_proj
v_proj v_proj
o_proj o_proj
rotary_emb rotary_emb

parent q_proj

parent k_proj

parent v_proj

parent o_proj

parent rotary_emb

parent mlp
gate_proj gate_proj
up_proj down_proj
down_proj up_proj
act_fn act_fn

parent gate_proj


AssertionError: ('up_proj', 'down_proj')

In [61]:
output = model.model(input_ids)

In [65]:
isinstance(output[0], torch.Tensor)

True

In [5]:
lin = torch.nn.Linear(5, 5, device="cuda")
b = torch.randn(5, device="cuda")
with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
    out = lin(b)
    print(out.dtype)
    loss = out.norm()
    print(loss.dtype)
loss.backward()
print(lin.weight.grad.dtype)

torch.bfloat16
torch.float32
torch.float32


In [6]:
lin.weight.grad

tensor([[ 5.2246e-02, -2.7771e-03, -4.8340e-02, -9.6680e-02, -7.7637e-02],
        [ 5.5237e-03, -2.9182e-04, -5.0964e-03, -1.0193e-02, -8.1787e-03],
        [ 2.7148e-01, -1.4404e-02, -2.5195e-01, -5.0391e-01, -4.0430e-01],
        [-1.2012e-01,  6.3477e-03,  1.1084e-01,  2.2168e-01,  1.7871e-01],
        [-1.9141e-01,  1.0132e-02,  1.7676e-01,  3.5352e-01,  2.8516e-01]],
       device='cuda:0')

In [7]:
weight = lin.weight.clone()
weight_bf16 = weight.clone().to(torch.bfloat16)
weight.eq(weight_bf16.to(torch.float32)).all()

tensor(False, device='cuda:0')

In [ ]:
grad = lin.weight.grad.clone()
grad_bf16 = grad.clone().to(torch.bfloat16)
grad.eq(grad_bf16.to(torch.float32)).all()

tensor(True, device='cuda:0')

: 

In [4]:
# Creates some tensors in default dtype (here assumed to be float32)
a_float32 = torch.rand((8, 8), device="cuda")
b_float32 = torch.rand((8, 8), device="cuda")
c_float32 = torch.rand((8, 8), device="cuda")
d_float32 = torch.rand((8, 8), device="cuda")

with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    # torch.mm is on autocast's list of ops that should run in float16.
    # Inputs are float32, but the op runs in float16 and produces float16 output.
    # No manual casts are required.
    e_float16 = torch.mm(a_float32, b_float32)
    print(e_float16.dtype)
    # Also handles mixed input types
    f_float16 = torch.mm(d_float32, e_float16)
    print(f_float16.dtype)

# After exiting autocast, calls f_float16.float() to use with d_float32
g_float32 = torch.mm(d_float32, f_float16.float())

torch.bfloat16
torch.bfloat16


In [ ]:
class FullTensorCustomOptimizer(torch.optim.Optimizer):
    def init(self, params, lr=1e-3):
        defaults = dict(lr=lr)
        super().init(params, defaults)
        
        # Create global projection matrices
        self.global_proj_to_low = torch.randn(20, 100)  # for 100x100 -> 20x100
        self.global_proj_to_full = torch.randn(100, 20)  # for 20x100 -> 100x100

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                
                full_grad = p.grad
                state = self.state[p]
                grad_proj = torch.mm(state["proj"], full_grad)

                if len(state) == 0:
                    state["step"] = 0
                    state["exp_avg"] = torch.zeros_like(grad_proj)
                    state["exp_avg_sq"] = torch.zeros_like(grad_proj)

                ###
                # Normal AdamW update with grad_proj instead of p.grad to compute exp_avg and denom
                ### 
                update_proj = exp_avg / denom
                update = torch.mm(state["proj"].T, update_proj)

                p.add_(update, value=-step_size)

In [5]:
state_dict = torch.load("0-fsdp_ckpt.pt")

In [4]:
import torch
ddp = torch.tensor([ 1.8258e-06, -3.6452e-06,  1.5652e-06,  4.5530e-06,  1.6738e-05,
         5.8220e-06, -1.3755e-06, -1.8935e-06])
fsdp = torch.tensor([ 2.9776e-06, -5.9446e-06,  2.5525e-06,  7.4251e-06,  2.7296e-05,
         9.4945e-06, -2.2432e-06, -3.0879e-06])

In [5]:
ddp / fsdp

tensor([0.6132, 0.6132, 0.6132, 0.6132, 0.6132, 0.6132, 0.6132, 0.6132])

In [6]:
3163136 // 4

790784

In [7]:
790784

790784

In [8]:
8192128 * 4

32768512

In [9]:
4096000 * 2 * 4 + 512

32768512